In [22]:
!pip install surprise
from surprise import Dataset
from surprise.model_selection import train_test_split, cross_validate
from surprise.prediction_algorithms import BaselineOnly, SVD, SVDpp, NMF

Завантажуємо movielens-100k датасет (алгоритм запропонує завантажити його, якщо необхідно)

In [23]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=0.2)


Проведемо порівняння алгоритмів :

*   SVD
*   SVD++
*   NMF

Побудуємо моделі для кожного алгоритму і ознайомимося з їх результатами, щоб визначити їх ефективність.

Для розуміння додамо коротенький опис методів:



1.   SVD (Singular Value Decomposition): Один з класичних методів розкладу матриці з дійсними або комплексними числами. Є узагальненням власного розкладу матриці невід'ємно визначеної нормальної матриці (наприклад, симетричної матриці з додатними власними значеннями) на матрицю розміру
m x n як узагальнення полярного розкладу.
2.   SVD++: Цей алгоритм включає в себе додаткові функції, такі як здатність рекомендувати нові або менше оцінені елементи, що може покращити точність прогнозування.
3.   NMF (Non-Negative Matrix Factorization): Цей алгоритм використовує не негативну матричну факторизацію, яка може бути корисною, коли рейтинги є не від'ємними числами.

In [24]:
svd = SVD()
svdpp = SVDpp()
nmf = NMF()


models = [svd, svdpp, nmf]
for model in models:
    results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    print(f"{type(model).__name__}:")
    print("  RMSE:", results['test_rmse'].mean())
    print("  MAE:", results['test_mae'].mean())

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9467  0.9239  0.9371  0.9371  0.9388  0.9367  0.0073  
MAE (testset)     0.7468  0.7297  0.7392  0.7372  0.7387  0.7383  0.0054  
Fit time          1.72    1.82    1.59    1.64    2.56    1.87    0.36    
Test time         0.12    0.55    0.13    0.22    0.59    0.32    0.20    
SVD:
  RMSE: 0.9367079238306826
  MAE: 0.7383225451250169
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9177  0.9220  0.9177  0.9203  0.9198  0.9195  0.0016  
MAE (testset)     0.7182  0.7269  0.7199  0.7185  0.7211  0.7209  0.0032  
Fit time          30.87   29.73   29.66   29.11   29.75   29.83   0.57    
Test time         5.35    5.58    5.47    5.57    5.81    5.55    0.15    
SVDpp:
  RMSE: 0.9194800435090803
  MAE: 0.7209124265280125
Evaluating RMSE, MAE o

Розберемо метрики, які ми можемо спостерігати в результаті виконання коду:

*   RMSE Root Mean Squared Error (testset): Середньоквадратична помилка між реальними та передбаченими рейтингами на тестовому наборі даних. Чим менше значення, тим краще.
*   MAE Mean Absolute Error (testset): Середня абсолютна помилка між реальними та передбаченими рейтингами на тестовому наборі даних. Чим менше значення, тим краще.
*   Fit time: Час, необхідний для навчання моделі на кожному розділі даних. Зазвичай це вимірюється в секундах.
*   Test time: Час, необхідний для прогнозування рейтингів для тестового набору даних. Зазвичай це також вимірюється в секундах.

Потім ознайомлюємося з усередненими значеннями RMSE та MAE зібраними на п'яти різних розділах даних для кожного із дослуджуваних методів .

Додамо також для порівняння Baseline модель. Це є простим методом прогнозування рейтингів, що базується на середній оцінці кожного користувача та середній оцінці для кожного предмета (фільму).

In [25]:
baseline = BaselineOnly()

results = cross_validate(baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print(f"{type(model).__name__}:")
print("  RMSE:", results['test_rmse'].mean())
print("  MAE:", results['test_mae'].mean())

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9554  0.9387  0.9411  0.9370  0.9505  0.9446  0.0072  
MAE (testset)     0.7545  0.7439  0.7465  0.7447  0.7547  0.7489  0.0048  
Fit time          0.49    0.46    0.62    0.68    0.39    0.53    0.11    
Test time         0.09    0.49    0.20    0.21    0.10    0.22    0.15    
NMF:
  RMSE: 0.9445568276808871
  MAE: 0.7488705491658902


**Висновок:**

Ознайомившись з результатами можемо спостерігати, що методи SVD та SVD++ показали кращі результати від звичайного методу рогнозування за середніми числами Baseline.

SVD++ показав найкращий результат, тим не менш - він потребує більшого часу для навчання моделі і прогнозування рейтингів.